# Intro to Flyte 2

To run this tutorial you'll need to be either an exsisting Union.ai user with Flyte v2, or [sign up for the free Beta access](https://www.union.ai/beta)!

Flyte 2.0



## Setup

if you're running this notebook locally we suggest creating a virtual environment and installing the packages locally with uv. You can follow the instructions on the README.

- uv venv .venv --python=python3.11
- source .venv/bin/activate
- uv pip install flyte>=0.2.0b21 --prerelease=allow

If you're running this in google colab run the two setup cells below:

In [2]:

!uv pip install flyte>=0.2.0b21 --prerelease=allow

zsh:1: 0.2.0b21 not found


##### Flyte Config
if you're running this locally remove `--auth-type headless\`

In [ ]:
!flyte create config \
    --endpoint demo.hosted.unionai.cloud \
    --auth-type headless\
    --builder remote \
    --domain development \
    --project flytesnacks

## 👋 Hello Flyte Tasks

In [ ]:
import flyte

env = flyte.TaskEnvironment(
    name="hello_flyte_v2",
    resources=flyte.Resources(cpu=1, memory="250Mi"),
)


@env.task
def extract_features(user_input: str) -> dict:
    clean = user_input.strip().lower()
    return {
        "text": clean,
        "length": len(clean),
        "word_count": len(clean.split()),
        "has_numbers": any(c.isdigit() for c in clean)
    }

@env.task
def validate_features(features: dict) -> bool:
    return features["length"] > 5 and features["word_count"] >= 2

@env.task
def prepare_training_data(raw_inputs: list[str]) -> dict:
    if len(raw_inputs) < 5:
        raise ValueError(f"Need at least 5 samples, got: {len(raw_inputs)}")

    features = list(flyte.map(extract_features, raw_inputs))
    valid_flags = list(flyte.map(validate_features, features))

    return {
        "total_samples": len(features),
        "valid_samples": sum(valid_flags),
        "avg_length": sum(f["length"] for f in features) / len(features),
        "ready_for_training": sum(valid_flags) >= len(features) * 0.8
    }

# Sample user reviews/comments for sentiment analysis training
sample_inputs = [
    "  This product is amazing! I love it.  ",    # valid: >5 chars, 2+ words
    "Great quality and fast shipping",             # valid: >5 chars, 2+ words
    " Bad ",                                       # invalid: too short
    "okay",                                        # invalid: <2 words
    "The delivery was delayed by 3 days",         # valid: >5 chars, 2+ words, has numbers
    "Excellent customer service team",             # valid: >5 chars, 2+ words
    "Perfect for my home office setup",           # valid: >5 chars, 2+ words
    "meh"                                          # invalid: too short, 1 word
]

flyte.init_from_config(".flyte/config.yaml")
execution = flyte.run(prepare_training_data, raw_inputs=sample_inputs)
print(f"Execution: {execution.name}")
print(f"URL: {execution.url}")
# Click on signin link to auth the first workflow 👇


#### Run tasks locally

In [ ]:
flyte.init() #overwrite flyte init from config
execution = flyte.run(prepare_training_data, raw_inputs=sample_inputs)
print(f"Execution: {execution.name}")
print(f"URL: {execution.url}")
#todo: output local info